In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pyodbc as pd

server = '<Host Name>'
database = 'covid-19-john-hopkins'
username = 'SQLID'
password = 'SQLPassword'

cnxn = pd.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server +
                     ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password +
                      ';TRUSTED_CONNECTION=yes;', autocommit=True)
cursor = cnxn.cursor()

data = cursor.execute("""
select top(10) Country_Region, Total_Confirmed from 
[dbo].[vw-case-per-month] where month = 6 and year = 2021 order by Total_Confirmed desc;
""").fetchall()

print(data)

country_region, total_confirmed = [], []

for tup in data :
    country, cases = tup
    country_region.append(country)
    total_confirmed.append(cases)

print(2021, 6, country_region, total_confirmed, sep = '\n\n')

cursor.close()
cnxn.close()

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.P("Colour:"),
    dcc.Dropdown(
        id="dropdown",
        options=[
            {'label': x, 'value': x}
            for x in country_region
        ],
        value='Gold',
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("dropdown", "value")])
def display_color(color):
    fig = go.Figure(
        data=go.Bar(x=country_region, y=total_confirmed, marker_color=color))
    return fig

app.run_server(debug=False)